In [4]:
from tqdm.auto import tqdm
import glob

In [5]:
%config Completer.use_jedi = False

In [6]:
%load_ext autoreload
%autoreload 2

In [8]:
import pandas as pd
pd.options.display.max_rows = 100

In [10]:
from tokenize_code import tokenizer_py
path = 'Lib/'
tokens = []
files = glob.glob(path+'/**/*.py', recursive=True)

def do(file):
    global tokens
    try:        
        out = tokenizer_py(file=file)
        tokens += out
    except Exception as e:
        print(e, file)

pbar = tqdm(files)
for file in pbar:
    pbar.set_description(f'{file}')
    do(file)

  0%|          | 0/952 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
len(tokens)

1847674

In [6]:
from collections import Counter
import pickle 

In [12]:
with open('lib_tokens.pkl', 'wb') as f:
    pickle.dump(tokens, f)

In [14]:
counter_lib = Counter(tokens)

In [15]:
lib_set = set(tokens)

In [17]:
len(lib_set)

63007

In [7]:
with open('lib_tokens.pkl', 'rb') as f:
    tokens = pickle.load( f)

In [11]:
import re
import pandas as pd
datasets = [[]]
file_name = 'python_text.py'

with open(file_name, 'r', encoding='utf-8') as f:
  #my_dict = {"description":[],"code":[]}
  for line in f:
    if line.startswith('#'):
      comment = line.split('\n#')
      if datasets[-1] != []:
        # we are in a new block
        datasets.append(comment)
    else:
      stripped_line = line#.strip()
      if stripped_line:
        datasets[-1].append(stripped_line)


[autoreload of tqdm.utils failed: Traceback (most recent call last):
  File "C:\Users\Ram\Miniconda3\envs\torch\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Ram\Miniconda3\envs\torch\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Ram\Miniconda3\envs\torch\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\Ram\Miniconda3\envs\torch\lib\site-packages\IPython\extensions\autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "C:\Users\Ram\Miniconda3\envs\torch\lib\site-packages\IPython\extensions\autoreload.py", line 280, in update_instances
    ref.__class__ = new
  File "C:\Users\Ram\Miniconda3\envs\torch\lib\site-packages\tqdm\utils.py", line 91, in __setattr__
    return setattr(self._wrapped, name, value)
TypeError: __class__ 

In [12]:
raw_data = {'Description' : [re.sub(r"^#(\d)*",'',x[0]).strip() for x in datasets], 'Code': [''.join(x[1:]) for x in datasets]}
df = pd.DataFrame(raw_data, columns=["Description", "Code"])

In [13]:
df = df.drop(0)

In [14]:
df

,Description,Code
1,write a python function to add two user provid...,"\n\ndef add_two_numbers(num1, num2):\n sum ..."
2,write a program to find and print the largest ...,\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 ...
3,write a program to find and print the smallest...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <=...
4,Write a python function to merge two given lis...,"\n\ndef merge_lists(l1, l2):\n return l1 + ..."
5,Write a program to check whether a number is p...,num = 337\n\n\nif num > 1:\n for i in range(...
...,...,...
4365,Write a program to print bit wise OR of two nu...,"a=60\nb=13\n\n\nc=a | b\nprint(""OR"", c)\n\n\n\n\n"
4366,Write a program to print bit wise XOR of two n...,"a=60\nb=13\n\n\nc=a ^ b\nprint(""XOR"", c)\n\n\n..."
4367,Write a program to calculate Binary Ones Compl...,"a=60\n\n\nc=~a\nprint(""Binary Ones Complement""..."
4368,write a program to Binary Left Shift a number,"c=a << 2\nprint(""Binary Left Shift"", c)\n\n\n\n\n"


In [15]:
from tokenize_code import tokenize_r

In [16]:
a = df.iloc[1]['Code']
a

"\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= num2) and (num1 >= num3):\n   largest = num1\nelif (num2 >= num1) and (num2 >= num3):\n   largest = num2\nelse:\n   largest = num3\nprint(f'largest:{largest}')\n\n\n"

In [17]:
df_final1 = pd.DataFrame()
for _, row in tqdm(df.iterrows()):
    code = tokenize_r(code=row['Code'])
    
    if code:
        row['Code'] = ''.join(a for a in code)
        df_final1 = df_final1.append(row)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
df_final1.columns

In [ ]:
df_final1=df_final1[['Description', 'Code']]

In [ ]:
df_final1

In [ ]:
df_final1[df_final1['Description'].str.contains('function')]

In [ ]:
df_final1['len'] = df_final1['Code'].str.len()
df_final1 = df_final1.drop(df_final1[df_final1['len'] == 0].index)
df_final1 = df_final1[df_final1['len'] < 251]

In [ ]:
df_final1.to_pickle('final_code_samples.pkl', protocol=4)
df_final1.to_csv('final_code_samples.csv')

In [ ]:
# Dividing the data into train and validation dataset

train_df = df_final1.sample(frac = 0.85) 
  
# Creating dataframe with rest of the 20% values 
valid_df = df_final1.drop(train_df.index)


In [ ]:
train_df

In [ ]:
valid_df

In [ ]:
train_df['Code']=train_df['Code'].str.replace('\t', '<TAB>')
train_df['Code']=train_df['Code'].str.replace('\n', '<NL>')
train_df['Code']=train_df['Code'].str.replace(' ', '<SPC>')

valid_df['Code']=valid_df['Code'].str.replace('\t', '<TAB>')
valid_df['Code']=valid_df['Code'].str.replace('\n', '<NL>')
valid_df['Code']=valid_df['Code'].str.replace(' ', '<SPC>')

In [ ]:
train_df[train_df['Code'].str.contains('\t')]

In [ ]:
valid_df[valid_df['Code'].str.contains('\t')]


In [ ]:
train_df = train_df[['Description', 'Code']]
valid_df = valid_df[['Description', 'Code']]

train_df.to_csv('train.tsv', index=False, sep='\t')
valid_df.to_csv('valid.tsv', index=False, sep='\t')

In [ ]:
df_final1 = df_final1.drop(df_final1[df_final1['len'] == 0].index)

In [ ]:
df_final2 = df_final1.sort_values(by='Description', key=lambda x:x.str.len())

In [ ]:
df_final2.sort_values(by='len').iloc[100:200]

In [ ]:
df_final2.iloc[:100]